In [1]:
import openrouteservice as ors
import folium
from pandas import DataFrame
import pandas as pd
import math
import time

In [2]:
client = ors.Client(key='5b3ce3597851110001cf6248e87222039de944878943e770cf957d7d')

In [3]:
raw_dir = '../data/raw/'
cur_dir = '../data/curated/'

In [5]:
rent = pd.read_csv("../rent_data/renew(15)")
rent = rent[:10].reset_index()
school = pd.read_csv(f"{cur_dir}school_df_21_preprocessed.csv")

In [6]:
# type the coordinates u need
# coordinates = 
"""
function: draw route map, return route
param datas : map, route
"""
def mapping(coordinates, client):
    map = folium.Map(location=[37.50, 144.58], tiles='cartodbpositron', zoom_start=13)
    route = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson',
        validate=False,
        )
    folium.PolyLine(locations=[list(reversed(coord)) for coord in route['features'][0]['geometry']['coordinates']]).add_to(map)
    return route, map


In [7]:

"""
function: calculate coordinates' distance
param datas : durations, distances
"""
def cal_distance(coordinates, client):
    matrix = client.distance_matrix(
    locations=coordinates,
    profile='driving-car',
    metrics=['distance', 'duration'],
    validate=False,
    )
    #time.sleep(2)
    return matrix['durations'], matrix['distances']
# the unit of distance is m, the unit of durations is seconds


In [23]:

"""
function: The number of times the code needs to run to the one house and all train stations.
param datas : times_i(corresponding i which is in function), times_function
"""
#as api only apply 3500 times each time
def times(len):
    times_i = int(math.sqrt(4))-1
    times = math.ceil(len/times_i)
    times_function = math.ceil((len+times)/times_i)
    return times_i, times_function



In [24]:
coordinates = []
coor = []
for j in range(10):
    coordinates.append([school["X"][j], school["Y"][j]])

print(cal_distance(coordinates, client))
#print(coordinates)
for k in range(5):
    coor.extend(coordinates[k*times(len(coordinates))[0]:(k+1)*times(len(coordinates))[0]])
    print(cal_distance(coor, client))
    #print(coordinates[k*times(len(coordinates))[0]:(k+1)*times(len(coordinates))[0]])


([[0.0, 20555.47, 17492.99, 15107.9, 10006.96, 13609.3, 12354.87, 14617.95, 20620.65, 7867.51], [20530.59, 0.0, 9058.34, 14925.26, 12978.31, 7625.33, 10110.5, 7807.89, 22130.72, 13085.37], [17464.36, 9050.9, 0.0, 9655.45, 9650.76, 8124.4, 5506.29, 3578.14, 16860.92, 10019.14], [15278.62, 15087.0, 9724.8, 0.0, 5623.9, 8140.83, 6543.62, 8806.7, 7422.77, 7833.4], [10062.78, 13011.73, 9726.46, 5613.33, 0.0, 6065.56, 4588.34, 6851.42, 12818.8, 2617.56], [13642.54, 7683.17, 8141.32, 8037.21, 6090.26, 0.0, 3222.45, 4842.28, 15242.67, 6197.32], [12427.92, 10187.66, 5482.58, 6455.46, 4614.32, 3241.49, 0.0, 2607.54, 13660.92, 4982.7], [14650.66, 7811.68, 3498.83, 8678.19, 6837.06, 4852.18, 2692.59, 0.0, 15883.66, 7205.44], [20649.11, 22230.75, 16868.55, 7390.88, 12767.65, 15284.58, 13687.37, 15950.45, 0.0, 14977.15], [8000.58, 13209.22, 10146.74, 7761.64, 2660.71, 6263.05, 5008.62, 7271.7, 14967.11, 0.0]], [[0.0, 461953.0, 396237.13, 339189.84, 213709.8, 293650.16, 269775.84, 328039.09, 453876.3

ApiError: 400 ({'error': {'code': 6003, 'message': "Parameter 'locations' has incorrect value or format."}, 'info': {'engine': {'version': '6.7.0', 'build_date': '2022-02-18T19:37:41Z'}, 'timestamp': 1663471280270}})

In [23]:
"""
function: Find the nearest site to rent 
param datas : rent_address, stop_name, min_distance, min_duration
"""
def get_coordinates(rent, school):
    rent_address = []
    school_name = []
    min_distance = []
    min_duration = []
    for i in range(len(rent)):
        coordinates = []
        distances = []
        durations = []
        for j in range(len(school)):
            coordinates.append([school["X"][j], school["Y"][j]])
        for k in range(times(len(school))[1]):
            coor_part = []
            coor_part.append([rent["longitude"][i], rent["latitude"][i]])
            coor_part.extend(coordinates[k*times(len(coordinates))[0]:(k+1)*times(len(coordinates))[0]])
            #matrix = cal_distance(coor_part, client)
            distances = cal_distance(coor_part, client)[1]
            durations = cal_distance(coor_part, client)[0]
        index = distances.index(min(distances))
        rent_address.append(rent['listingSlug'][i])
        school_name.append(school['School_Name'][index])
        min_distance.append(distances[index])
        min_duration.append(durations[index])
    return rent_address, school_name, min_distance, min_duration


In [31]:

"""
function: put the required columns into a dataframe
param datas: dataframe
"""


data = {'rent_address': get_coordinates(rent, school)[0], 'school_name': get_coordinates(rent, school)[1], 'min_distance': get_coordinates(rent, school)[2], 'min_duration': get_coordinates(rent, school)[3]}
df = pd.DataFrame(data)

df.to_csv(f"{cur_dir}/distances_durations_school21.csv", index=False)

/home/jnniu/miniconda3/envs/group55/lib/python3.8/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/jnniu/miniconda3/envs/group55/lib/python3.8/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/jnniu/miniconda3/envs/group55/lib/python3.8/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/jnniu/miniconda3/envs/group55/lib/python3.8/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_count

Timeout: 